# Meghana Ravikumar
# Decision Trees and Random Forest

In [310]:
import csv
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import scipy.io
import math
import random
import itertools

## Following class is used to import and preprocess data 
### The census data is read line by line into a numpy matrix. The missing values are preprocessed by using a kNN approach (implemented partially through sklearn), and the categorical data is preprocessed and mapped into discrete categories. The descrete categories are used to make the data more generalizable and take less space in memory to process. The kNN approach was used to fill the variables to ensure that the missing data was accurately substituted. This allows for a better fitting decision tree and more accurate predictions. 
### The spam data was processed through sklearn by loadmat. There were no missing values in the spam dataset. In order to improve accuracy of predictions and develop a better fitting tree, the spam data is binarized (through the sklearn.binarize package) and more features were added. Specifically the following features were added to featurize.py:  'monies', 'viagra', 'unsubscribe', 'nigeria', 'supplement', 'monetary', 'fund', 'warn', 'porn', 'staff', 'attach', '%', '-', '.', '?', '^', '@', '\'
### After a manual examination of many of the spam emails, these features were found to dominate several of these emails. In order to increas accuracy in prediction, the process of feature extraction can be automated to mine for the overall top 10 features in all ham and spam email. This mining will allow for a wider variety of words to be selected that span both categories of emails, leading to a less biased decision tree. 



In [556]:
class importData(object):
    def __init__(self,myPath, categorical_data, continuous_data):
        self.columns = None
        self.path = myPath
        self.data = None
        self.missing = None #dictionary
        self.filledValues = None
        self.transformedData = None
        self.categorical_data = categorical_data
        self.continuous_data = continuous_data

    def readingCensusData(self):
        census_matrix = np.array([])
        with open(self.path,"rb") as f:
            spamreader = csv.reader(f, delimiter = ",")
            for row in spamreader:
                if len(census_matrix)==0:
                    census_matrix = np.array(row).reshape(1,len(row))
                else:
                    data = np.array(row).reshape(1,len(row))
                    census_matrix = np.vstack((census_matrix,data))
        self.data = census_matrix
        return (census_matrix)

    def readingHousingData(self):
        spam_dataset = scipy.io.loadmat(self.path)
        spam_dataset_labels = spam_dataset['training_labels'][0]
        spam_dataset_train = spam_dataset['training_data']
        spam_dataset_test = spam_dataset['test_data']
        return(spam_dataset_labels,spam_dataset_train,spam_dataset_test)

    def transformingData(self):
        transformedData = self.data
        categorial_data = self.categorical_data
        continuous_data = self.continuous_data
        missing = np.where(transformedData == "?")
        missing_values = {}

        for i in range(len(missing[0])):
            if missing[1][i] not in missing_values.keys():
                missing_values[missing[1][i]]=[missing[0][i]]
            else:
                missing_values[missing[1][i]].append(missing[0][i])

        for c in categorial_data:
            transformedData[:,c][1:]= preprocessing.LabelEncoder().fit_transform(transformedData[:,c][1:])

        for t in continuous_data:
            transformedData[:,t][1:] = transformedData[:,t][1:]
        self.missing = missing_values
        self.transformedData = transformedData
        
        return (self.transformedData, self.missing)
    
    def convertingToFloat(self):
        '''method takes in a numpy array and converts everything to a float'''
        transformedData = self.transformedData
        convertedData = np.array([])
        for col in range(0,transformedData.shape[1]):
            currData = transformedData[:,col][1:].astype(float).reshape(len(transformedData[:,col][1:]),1)
            if len(convertedData)==0:
                convertedData = currData.astype(float)
            else:
                convertedData = np.hstack((convertedData,currData))
        return convertedData

    def prep_kNN(self):
        convertedData = self.convertingToFloat()
        missing_values = self.missing

        for column_value in missing_values.keys():
            rowValues = missing_values[column_value]
            testMat = np.array([])
            trainingData = np.array([])
            trainingLabels = np.delete(convertedData[:,column_value],rowValues+[0])

            #iterate through matrix and split into test and training set
            for ind in range(1,convertedData.shape[0]):
                currData = np.hstack((convertedData[ind,0:column_value],
                                  convertedData[ind,column_value+1:]))
                if ind in rowValues:
                    if len(testMat) == 0:
                        testMat = currData
                    else:
                        testMat = np.vstack((testMat, currData))
                else:
                    if len(trainingData)==0:
                        trainingData= currData
                    else:
                        trainingData = np.vstack((trainingData, currData))
                    
            predicted = self.kNN(testMat, trainingData, trainingLabels) 
            #np.array -- indexing should be the same as given rowValues
            #need to index transformedData to put in new values
            for r in range(len(rowValues)):
                convertedData[rowValues[r], column_value] = predicted[r]
        self.filledValues = convertedData
        return self.filledValues 

    def kNN(self,testMat, trainingData, trainingLabels): #input is a list of test, training, and label set
        knn = KNeighborsClassifier()
        knn.fit(trainingData, trainingLabels)
        predictions = knn.predict(testMat)
        return predictions

In [410]:
def splittingData(spam_data_train):
    numVal = int(spam_data_train.shape[0]/3)
    numTrain = spam_data_train.shape[0] - int(numVal)
    valIndices = random.sample(range(0,spam_data_train.shape[0]), numVal)
    validation_data = np.zeros((numVal,spam_data_train.shape[1]))
    training_data = np.zeros((numTrain,spam_data_train.shape[1]))
    valInd = 0
    trainInd = 0
    for spam_ind in range(spam_data_train.shape[0]):
        if spam_ind in valIndices:
            validation_data[valInd]= spam_data_train[spam_ind,:]
            valInd = valInd +1
        else:
            training_data[trainInd] = spam_data_train[spam_ind,:]
            trainInd = trainInd + 1
    return (validation_data,training_data)

In [553]:
def importing_censusData():
    census_object = importData("./census_data/train_data.csv", [1,3,5,6,7,8,9,13], [0,2,4,10,11,12] )
    census_matrix = census_object.readingCensusData()
    transformedData, missing_values = census_object.transformingData()
    filled_transformed = census_object.prep_kNN()
    filled_transformed_permuted = np.random.permutation(filled_transformed[1:])
    (census_train, census_validate) = splittingData(filled_transformed_permuted)
    return (census_train, census_validate)

In [554]:
def importing_censusTest():
    census_object2 = importData("./census_data/test_data.csv", [1,3,5,6,7,8,9,13], [0,2,4,10,11,12] )
    census_matrix2 = census_object2.readingCensusData()
    transformedData2, missing_values2 = census_object2.transformingData()
    filled_transformed2 = census_object2.prep_kNN()
    return (filled_transformed2)

In [568]:
def import_processingSpam():
    (spam_dataset_labels,spam_dataset_train,
     spam_dataset_test) = importData("./spam-dataset/spam_data.mat",[],[]).readingHousingData()
    spam_dataset_labels = spam_dataset_labels.reshape(spam_dataset_labels.shape[0],1)
    spam_dataset_train_labels = np.concatenate((spam_dataset_train, spam_dataset_labels),axis=1)
    spam_dataset_permutation = np.random.permutation(spam_dataset_train_labels)
    spam_data_permutation = preprocessing.binarize(spam_dataset_permutation)
    (spam_dataset_train, spam_dataset_validate) = splittingData(spam_dataset_permutation)
    return (spam_dataset_train, spam_dataset_validate, spam_dataset_test)

In [532]:
(census_train, census_validate) = importing_censusData()

in prep knn
(32724, 15)
in knn
in knn
in knn


In [534]:
census_test = importing_censusTest()

in prep knn
(16118, 14)
in knn
in knn
in knn


In [569]:
(spam_dataset_train, spam_dataset_validate, spam_dataset_test)= import_processingSpam()

## Descision Tree and Node Classes

### The following Decision Tree and Node classes are used to build and train a decision treen and train/ predict labels using this model. The tree is built by beginning with the whole data set of either Spam or Census. It then recursively builds the tree by calculating the minimum impurity/weighted entropy resulting from a chosen split for each depth. In order to determine if a node is a leaf node, three types of thresholds are used. The thresholds set in place are: maximum depth for the tree, composition of the current node, and the number of data points in the current node. The maximum depth was tinkered with to find that a depth of 12 worked best for Spam data, and a depth of 10 worked best for Census data. If a node was found to have 5 or less data points, it was deemed as a leaf node. This threshold was put in place to prevent over-fitting. Furthermore, each node was checked for percent composition of class 0 and class 1 data points. If the composition of one of the two classes was found to be less than a set threshold (either 0.001 or 0.0001), the node was deemed a leaf. If the subset of the original data at the current node was found to hit one of these thresholds, the node is a leaf. 
### The thresholds for categorical and continuous data were determined in two different ways. If a feature contained categorical variables, the threshold looked for the number of data points in the data that had feature values that were equal to the variable. (i.e. data[datapoint, feature]== variable). Although this doesn't take into account all possible combinations at each iteration, it still allows the data to be split over other variables in the feature deeper into the tree, essentially taking into account all the combinations of data. If a feature's variables were continuous, the variables were binned randomly for each split in the tree. The random binning was  chosen to reduce the number of splitting points/thresholds the continuous feature may have. This process was performed for each variable within each feature and each feature for the data. Data points that satisfy the threshold became the left node, and data points that don't satisfy the threshold became the right node.
### The Node Class is used for storing the data gathered at each split. 


In [658]:
class DecisionTree(object):

    def __init__(self, master_data, categorial_data, continuous_data, max_depth,
                 min_impurity, features_allowed):
        self.master_data = master_data
        self.categorial_data = categorial_data
        self.continuous_data = continuous_data
        self.max_depth = max_depth
        self.min_impurity = min_impurity
        self.allCombinations_categorical = None
        self.rootNode = None
        self.features_allowed = features_allowed

    def impurity(self,left_label_hist, right_label_hist):
        ''' Method calculates the impurity for the given dataset. Has two possible classes --> 0,1
        Input is a dictionary of {key,value} --> {class,freq}'''
        
        total_left = sum(left_label_hist.values())
        total_right = sum(right_label_hist.values())
        
        if (total_left + total_right) == 0:
            return float(0) 
            
        if total_left == 0:
            left_entropy = 0
        else:
            prbL_0 = (left_label_hist[0])/float(total_left)
            prbL_1 = (left_label_hist[1])/float(total_left)
            left_entropy = -1*sum([prbL_0*np.log2(prbL_0), prbL_1*np.log2(prbL_1)])
            if np.log2(prbL_0)== -float("inf"):
                left_entropy = float(0)
            if np.log2(prbL_1)== -float("inf"):
                left_entropy = float(0)
            
        if total_right == 0:
            right_entropy = 0
        else:
            prbR_0 = (right_label_hist[0])/float(total_right)
            prbR_1 = (right_label_hist[1])/float(total_right)
            right_entropy = -1*sum([prbR_0*np.log2(prbR_0), prbR_1*np.log2(prbR_1)])
            if np.log2(prbR_0)== -float("inf"):
                right_entropy = float(0)
            if np.log2(prbR_1)== -float("inf"):
                right_entropy = float(0)

        impurity = ((total_left*left_entropy)+(total_right*right_entropy))/(total_left+total_right)
        return impurity 
    
    def composition(self,data):
        '''This method will calculate the entropy for a given set of data. '''
        master_data = self.master_data
        num_0 = 0
        num_1 = 0
        for row_ind in data:
            num = int(master_data[row_ind,master_data.shape[1]-1])
            if num == 0:
                num_0 = float(num_0 +1)
            else:
                num_1 = float(num_1+1)
        
        return min(num_0/(num_0+num_1),num_1/(num_0+num_1))
    
    def entropy(self,data):
        ##calculate entropy for current parent node
        master_data = self.master_data
        num_0 = 0
        num_1 = 0
        for row_ind in data:
            num = int(master_data[row_ind,master_data.shape[1]-1])
            if num == 0:
                num_0 = float(num_0 +1)
            else:
                num_1 = float(num_1+1)
        total = num_0 + num_1
        if num_0 == 0:
            return -1*(num_1/total)*np.log2(num_1/total)
        if num_1 == 0:
            return -1*(num_0/total)*np.log2(num_0/total)
        return -1*sum([(num_0/total)*np.log2(num_0/total), (num_1/total)*np.log2(num_1/total)])
        
        
    def segementer(self,data):
        '''Method will look through all possible splits, calcualte impurity, and return the combination
        that results in min impurity.'''
        
        master_data = self.master_data
        label = master_data[:,master_data.shape[1]-1]
        categories = self.categorial_data
        continuous = self.continuous_data
        curr_impurity = math.pow(10,10)
        toRtn_left = None
        toRtn_right = None
        min_split = None

        if self.features_allowed < self.master_data.shape[1]-1: 
            feature_indices = np.random.choice(range(self.master_data.shape[1]-1), size = self.features_allowed, replace = True)
        else:
            feature_indices = range(self.master_data.shape[1]-1)


        ##iterate through feature indices (column indices) to get the splits for each column
        for col_ind in feature_indices:
            if col_ind in categories:
                categorical = True
                poss_splits = np.unique(master_data[:,col_ind])
            else:
                categorical = False
                if len(np.unique(master_data[:,col_ind])) >=4:
                    poss_splits = np.sort(random.sample(np.unique(master_data[:,col_ind]), 4))
                else:
                    poss_splits = np.unique(master_data[:,col_ind])
            for pos in poss_splits:
                left_count = {0:0,1:0}
                right_count = {0:0,1:0}
                left_mat = []
                right_mat = []
                for row_ind in data:
                    if categorical == True:
                        if (master_data[row_ind,col_ind] == pos) and (label[row_ind]==0):
                            left_count[0] = left_count[0]+1
                            left_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] == pos) and (label[row_ind]==1):
                            left_count[1]=left_count[1]+1
                            left_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] != pos) and (label[row_ind]==0):
                            right_count[0] = right_count[0]+1
                            right_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] != pos) and (label[row_ind]==1):
                            right_count[1]=right_count[1]+1
                            right_mat.append(row_ind)
                    else:
                        
                        if (master_data[row_ind,col_ind] <= pos) and (label[row_ind]==0):
                            left_count[0] = left_count[0]+1
                            left_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] <= pos) and (label[row_ind]==1):
                            left_count[1]=left_count[1]+1
                            left_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] > pos) and (label[row_ind]==0):
                            right_count[0] = right_count[0]+1
                            right_mat.append(row_ind)
                        if (master_data[row_ind,col_ind] > pos) and (label[row_ind]==1):
                            right_count[1]=right_count[1]+1
                            right_mat.append(row_ind)
                calc_impurity = self.impurity(left_count, right_count)
                if (calc_impurity <= curr_impurity):
                    curr_impurity = calc_impurity 
                    min_split = (col_ind, pos)
                    toRtn_left = left_mat
                    toRtn_right = right_mat
        return (min_split, toRtn_left, toRtn_right)
    
    
    def train(self,data,n):
        '''This method recursively builds a decision tree and trains the split at each node.
        Input - data matrix and vector of labels. Will increment depth here. 
        Nodes carry the following information: left, right, current split'''
        #need to set up root node and check for entropy of current node AFTER each split -- if min has been reached, DON'T split
        #OR if min-impurity is reached from creating a split -- stop
        #data = range(self.master_data.shape[0])
        self.rootNode = self.growTree(data,0,n)
        return self

    def growTree(self, data, depth,n):
        
        currImpurity = self.composition(data)

        if (currImpurity <= self.min_impurity) or (depth >= self.max_depth) or (len(data)<= n):
            currLabel = self.set_label(data)
            return Node(left = None, right =None, split_rule = None, label =currLabel)            
        else:
            (curr_split, left_matrix, right_matrix) = self.segementer(data)
            if len(right_matrix) == 0:
                ## create leaf node with the majority value of the left matrix
                currLabel = self.set_label(left_matrix)
                return Node(left = None, right= None, split_rule = None, label = currLabel)
            if len(left_matrix) == 0:
                currLabel = self.set_label(right_matrix)
                return Node(left = None, right= None, split_rule = None, label = currLabel)
            else:
                return Node(left = self.growTree(left_matrix, depth+1, n), 
                            right = self.growTree(right_matrix, depth+1, n), split_rule = curr_split,
                            label= None)

    def predict(self,data):
        '''This method will predict the labels for new test data matrices. Data is inputted as a numpy matrix. '''
        predicted_values = []
        for row_ind in range(data.shape[0]):
            currNode = self.rootNode
            depth = 0
            toRtn = ''
            while (currNode.label == None):
                (colNum, num_range) = currNode.split_rule #tupule (feature column number, subset list/integer)
                toRtn = toRtn + str(colNum) + " " + str(num_range)
                if colNum in self.categorial_data:
                    if data[row_ind, colNum] == num_range:
                        currNode = currNode.left
                        toRtn = toRtn + " "+ 'left'
                    else:
                        currNode = currNode.right
                        toRtn = toRtn + " "+ 'right'
                else:
                    if data[row_ind, colNum] <= num_range:
                        currNode = currNode.left
                        toRtn = toRtn + " "+ 'left'
                    else:
                        currNode = currNode.right
                        toRtn = toRtn + " "+ 'right'
                
                if row_ind == 1:
                    print toRtn
                depth = depth + 1
           
            predicted_values.append(currNode.label)

        return predicted_values

    def set_label(self, data):
        '''This method recognizes leaf nodes and returns the majority label within the node. 
        A node is considered a leaf node if the tree has reached  a maximum depth or if the tree has reached a
        minimum impurity threshold. This shall be indicated by no split rule.'''
        master_data = self.master_data
        [num_0,num_1] = [0,0]
        for row_ind in data:
            num = int(master_data[row_ind,master_data.shape[1]-1])
            if num == 0:
                num_0 = num_0 +1
            else:
                num_1 = num_1+1
        return [num_0,num_1].index(max([num_0,num_1]))

    
class Node(object):
    def __init__(self, left, right, split_rule, label):
        self.left = left #will indicate binary "yes"
        self.right = right #will indicate binary "no"
        self.split_rule = split_rule
        self.label = label

In [318]:
def accuracy(predicted_values, trueLabels):
    count = 0
    for v in range(len(predicted_values)):
        if predicted_values[v] == trueLabels[v]:
            count = count +1 
    return float(count)/len(predicted_values)

In [654]:
def spam_validation_test():
    spam_categorial_data = []
    spam_continuous_data = range(spam_dataset_train.shape[1]-1) #last column of matrix is labels
    tree = DecisionTree(spam_dataset_train,spam_categorial_data,
                    spam_continuous_data,12, 0.0001, spam_dataset_train.shape[1]-1)
    tree.train(range(spam_dataset_train.shape[0]), n=5)
    spam_pred = tree.predict(spam_dataset_validate)
    accuracy_validation = accuracy(spam_pred,spam_dataset_validate[:,spam_dataset_validate.shape[1]-1])
    spam_pred2 = tree.predict(spam_dataset_test)
    return accuracy_validation, spam_pred2

### The following tupule follow the splits that a radomly chosen data point encounters through the Spam decision tree. The first integer indicates the feature number/column number of the data matrix, the second integer indicates the variable of the given feature, 'left' indicates that the data point <= to the given feature&variable combination, 'right' indicates that the data point > the given feature&variable combination. 

In [655]:
validation_accuracy3, predictions_spam = spam_validation_test()

39 0.0 left
44 18.0 left
19 0.0 left
43 0.0 left
0 0.0 left
16 0.0 left
45 7.0 left
26 0.0 left
46 0.0 left
49 4.0 left
48 2.0 left
27 0.0 left
39 0.0 right
48 1.0 right
5 0.0 left
44 45.0 left
43 0.0 left
47 0.0 left
3 0.0 left
45 60.0 left


### Multiple parameters were manually tried to find those that ran quickly and created an accurate descision tree. In the end, the following parameters were found to be best for census data:  max depth = 12, minimum impurity/minimum percent composition = 0.0001. he validation accuracy for spam data is as follows and was found to be higher with a binarized data set:

In [632]:
validation_accuracy3


0.8442575406032483

### The following tupule follow the splits that a radomly chosen data point encounters through the Census decision tree. The first integer indicates the feature number/column number of the data matrix, the second integer indicates the variable of the given feature, 'left' indicates that the data point <= or == to the given feature&variable combination, 'right' indicates that the data point > or != the given feature&variable combination.

In [656]:
def census_validation_test():
    census_categorical_data = [1,3,5,6,7,8,9,13]
    census_continuous_data = [0,2,4,10,11,12] 
    censusTree = DecisionTree(census_train, census_categorical_data, census_continuous_data, 
                              10,0.001,census_train.shape[1]-1)
    censusTree.train(range(census_train.shape[0]), n=0)
    census_pred = censusTree.predict(census_validate)
    validation_acc = accuracy(census_pred,census_validate[:,census_validate.shape[1]-1])
    census_pred2 = censusTree.predict(census_test)
    #print census_pred2
    return validation_acc, census_pred2

In [657]:
validation_acc, census_predictions = census_validation_test()

5 2.0 left
10 4386.0 left
4 7.0 right
3 11.0 left
11 1741.0 left
0 33.0 right
6 4.0 right
0 65.0 left
12 25.0 right
0 35.0 right
5 2.0 right
10 4865.0 left
4 12.0 right
5 4.0 right
11 2205.0 left
12 48.0 left
6 4.0 right
9 1.0 right
6 1.0 right
0 34.0 right


### Multiple parameters were manually tried to find those that ran quickly and created an accurate descision tree. In the end, the following parameters were found to be best for census data:  max depth = 10, minimum impurity/minimum percent composition = 0.001. The validation accuracy for census is as follows:

In [ ]:
validation_acc

## Random Forest
### The following class is used to build a Random Forest. The random forest algorithm generates multiple descision trees trained on the test data for Spam/Census, validated on respective validation sets, and tested on their respective test sets. No new methods were built for Random Forest. The parameters for Random Forest are the same as for Descision Tree. Each tree in the forest is trained on the given test data, validated on the validation data, and tested/predicts on the test data. 
### In retrospect, bagging or ADAboost would have improved the accuracy of the random forest drastically, as each tree that is built would be weighted to a certain degree depending on its error rate. 

In [662]:
class RandomForest(object):
    ''' The following class is a subclass of Decision Tree. It uses the methods described in Decision tree to 
    perform bagging and implements Random Forests. The sampling for attribute bagging is done with sqrt(d) features.'''
    
    def __init__(self,master_data, categorial_data, continuous_data, max_depth, min_impurity,
                 max_sampling, feature_num, dataset_validate):
        self.master_data = master_data
        self.categorial_data = categorial_data
        self.continuous_data = continuous_data
        self.max_depth = max_depth
        self.min_impurity = min_impurity
        self.max_sampling = max_sampling
        self.feature_num = feature_num
        self.predicted_all = None
        self.dataset_validate = dataset_validate
    
    def bagging(self, data_pred, num_samples,min_num):
        master_data = self.master_data
        max_sampling = self.max_sampling
        predicted_all = np.array([])
        rootNode_all = {}
        i = 0
        while i <= max_sampling:
            data_forest = np.random.choice(range(master_data.shape[0]), size = num_samples, replace= True)
            predicted, rootNode = self.bagging_helper(data_pred,data_forest,min_num)
            
            if len(predicted_all) == 0:
                predicted_all = predicted
            else:
                predicted_all = np.vstack((predicted_all,predicted))
                
            if rootNode not in rootNode_all.keys():
                rootNode_all[rootNode]=1
            else:
                rootNode_all[rootNode]=rootNode_all[rootNode]+1
            i = i + 1
            
        self.predicted_all = predicted_all
        toRtn = np.apply_along_axis(self.averaging,0,predicted_all)
        return toRtn, rootNode_all
            
    def bagging_helper(self, data_pred, data_forest,min_num):
        '''This method will create Decision tree objects to run using the sampled data from bagging method.'''  
        tree = DecisionTree(self.master_data,self.categorial_data,
                    self.continuous_data,self.max_depth, self.min_impurity, self.feature_num)
        tree.train(data_forest, min_num)
        currRoot = tree.rootNode.split_rule
        predicted = tree.predict(data_pred) #label for each sample
        
        return predicted, currRoot
    
    def averaging(self,predicted):
        curr0 = np.where(predicted == 0)[0].size
        curr1 = np.where(predicted==1)[0].size
        return [curr0,curr1].index(max([curr0,curr1]))

        
        

In [665]:
def forest_census():
    census_categorical_data = [1,3,5,6,7,8,9,13]
    census_continuous_data = [0,2,4,10,11,12] 
    forest = RandomForest(master_data = census_train, categorial_data = census_categorical_data,
                    continuous_data = census_continuous_data, max_depth = 10, min_impurity= 0.001, 
                    max_sampling = 30, feature_num = int(math.sqrt(census_train.shape[1])),
                     dataset_validate = census_validate)
    bagged, rootNodes = forest.bagging(census_validate, census_train.shape[0], 0)
    acc = accuracy(bagged, census_validate[:,census_validate.shape[1]-1])
    bagged_test = forest.bagging(census_test, census_test.shape[0], 0)
    return (bagged_test,acc, rootNodes)

In [666]:
bagged_census, acc_census, rootNodes = forest_census()

### The following returns the ((feature, variable), frequency of split seen as root split in spam descision trees)

In [672]:
import operator
sorted_roots_census = sorted(rootNodes.items(), key=operator.itemgetter(1), reverse= True)[0:3]
print sorted_roots_census

[((5, 2.0), 8), ((7, 0.0), 5), ((3, 9.0), 4)]


In [667]:
def forest_spam():
    spam_categorial_data = []
    spam_continuous_data = range(spam_dataset_train.shape[1]-1) 
    forest = RandomForest(master_data = spam_dataset_train, categorial_data = spam_categorial_data,
                    continuous_data = spam_continuous_data, max_depth = 12, min_impurity= 0.0001, 
                    max_sampling = 30, feature_num = int(math.sqrt(spam_dataset_train.shape[1])),
                     dataset_validate = spam_dataset_validate)
    bagged, rootNodes = forest.bagging(spam_dataset_validate, spam_dataset_train.shape[0], 0)
    acc = accuracy(bagged,spam_dataset_validate[:,spam_dataset_validate.shape[1]-1])
    ##running trained forest on test set
    bagged_test = forest.bagging(spam_dataset_test, spam_dataset_test.shape[0],0)
    return (bagged_test, acc, rootNodes)

In [668]:
bagged_spam, acc_spam, rootNodes_spam = forest_spam()

### The following returns the ((feature, variable), frequency of split seen as root split in spam descision trees)

In [674]:
sorted_roots_spam = sorted(rootNodes_spam.items(), key=operator.itemgetter(1), reverse = True)[0:3]
print sorted_roots_spam

[((10, 0.0), 3), ((43, 0.0), 3), ((5, 0.0), 2)]


## Kaggle
### The kaggle score for a random forest on census data is: 0.762
### The kaggle score for a random forest on spam data is: 0.711
### The scores for random forest and a singular descision tree did not differ much. Furthermore, the validation scores for both datasets using random forest or a singular descision are both approximately 0.85. This indicates that the tree has over-fit the training data and the random forest can be made more powerful through boosting, ADAboost, cross-validation, etc.

In [365]:
def writeToCSV(fileName, predictions):
    import csv
    with open(fileName, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Id','Category'])
        shl = 1
        for pred_l in predictions:
            writer.writerow([shl,int(pred_l)])
            shl = shl + 1

In [543]:
writeToCSV('census_nonforest.csv',census_predictions)

In [644]:
writeToCSV('spam_nonforest.csv',predictions_spam)

In [641]:
writeToCSV('census_test_forest.csv',bagged_census)

In [642]:
writeToCSV('spam_test_forest2.csv',bagged_spam)